In [1]:
import os
import sys
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import glob
import csv
import pandas as pd
%matplotlib inline

import seaborn as sns
sns.set_style("whitegrid")

In [2]:
base_folder_location = '../scripts/Final/'

autoencoder_location = os.path.join(base_folder_location, 'Autoencoder/experiments/*')
autoencoder_fixmatch_location = os.path.join(base_folder_location, 'AutoencoderFixMatch/experiments/*')
fixmatch_location = os.path.join(base_folder_location, 'FixMatch/experiments/*')
mixmatch_location = os.path.join(base_folder_location, 'MixMatch/experiments/*')

In [8]:
def get_test_information(location, columns_names, columns_file_values):
    all_columns = []
    all_columns.extend(columns_names)
    all_columns.extend(columns_file_values)

    df = pd.DataFrame(columns=all_columns)
    
    for experiment in glob.glob(location):
        folder_name = os.path.basename(experiment)
        hyper_parameters = folder_name.split('_')
        
        test_summary_location = os.path.join(experiment, 'result_outputs/test_summary.csv')
        
        with open(test_summary_location, 'r') as csvfile:
            csvreader = csv.reader(csvfile)
            next(csvreader)
            values = next(csvreader)
            
            all_values = []
            all_values.extend(hyper_parameters)
            all_values.extend(values)
            
            df = df.append(dict(zip(df.columns, all_values)), ignore_index=True)
    
    return df

df_mixmatch = get_test_information(mixmatch_location, ['Model', 'Seed', 'Magnification', 'Amount of Labeled Images per subclass'], ['Test Acc', 'Test Loss'])

df_mixmatch["Test Loss"] = pd.to_numeric(df_mixmatch["Test Loss"])
df_mixmatch["Amount of Labeled Images per subclass"] = pd.to_numeric(df_mixmatch["Amount of Labeled Images per subclass"])
df_mixmatch["Test Acc"] = pd.to_numeric(df_mixmatch["Test Acc"])

df_fixmatch = get_test_information(fixmatch_location, ['Model', 'Seed', 'Magnification', 'Amount of Labeled Images per subclass'], ['Test Acc', 'Test Loss'])

df_fixmatch["Test Loss"] = pd.to_numeric(df_fixmatch["Test Loss"])
df_fixmatch["Amount of Labeled Images per subclass"] = pd.to_numeric(df_fixmatch["Amount of Labeled Images per subclass"])
df_fixmatch["Test Acc"] = pd.to_numeric(df_fixmatch["Test Acc"])

df_autofixmatch = get_test_information(autoencoder_fixmatch_location, ['Model', 'Seed', 'Magnification', 'Amount of Labeled Images per subclass'], ['Test Acc', 'Test Loss'])

df_autofixmatch["Test Loss"] = pd.to_numeric(df_autofixmatch["Test Loss"])
df_autofixmatch["Amount of Labeled Images per subclass"] = pd.to_numeric(df_autofixmatch["Amount of Labeled Images per subclass"])
df_autofixmatch["Test Acc"] = pd.to_numeric(df_autofixmatch["Test Acc"])

KeyboardInterrupt: 

In [4]:
# base_dense_net = pd.read_csv('../scripts/BaseDenseNet/binary_class/results_test.csv')

In [5]:
df_all = df_mixmatch.append(df_fixmatch).append(df_autofixmatch)

df_all['Model'] = df_all['Model'].str.replace('mixmatch', 'MixMatch')
df_all['Model'] = df_all['Model'].str.replace('fixmatch', 'FixMatch')
df_all['Model'] = df_all['Model'].str.replace('autoencoderFixMatch', 'Pre-trained FixMatch')

In [7]:
for magnification in ['40X', '100X', '200X', '400X']:
    df = df_all[df_all['Magnification'] == magnification]

    sns.lineplot(x="Amount of Labeled Images per subclass", y="Test Acc", 
                 hue="Model", style="Model",
                 markers=True, data=df).set_title(f'{magnification} Magnification')
    
    plt.savefig(f'Loss {magnification}.pdf')
    plt.clf()


<Figure size 432x288 with 0 Axes>